<a href="https://colab.research.google.com/github/Minh-A/For_model_reference/blob/main/Gene_Algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 유전자 알고리즘
* K-Means 클러스터링이 메인이 되어 자유전자를 생성

In [ ]:
import pandas as pd
import numpy as np
import os
import glob
import random

import warnings
warnings.filterwarnings("ignore")

train = glob.glob('/content/drive/MyDrive/Colab Notebooks/Auto_Analisys/file/*.csv')
temp = pd.read_csv(train[6], encoding='cp949', index_col= 'Unnamed: 0')
dataset = temp.iloc[0:40]
dataset = dataset.loc[::-1].reset_index(drop = True)  # 위아래 뒤집기(날짜 뒤집기)
da = dataset.iloc[:,[2,3,4,6,7,8]]

# K-Means 모델
from sklearn.cluster import KMeans
import matplotlib.pyplot  as plt

k = 3
# create model and prediction
model = KMeans(n_clusters=k, algorithm='elkan')
model.fit(da)
predict = pd.DataFrame(model.predict(da))
predict.columns=['predict']

final_da = pd.DataFrame(np.hstack((predict, da)))


In [ ]:
# 유전자 알고리즘 210117
# 첫 번째 모듈
import random
import copy

k_result = 2  # K-Means 결과 정하기
aaa =  copy.deepcopy(final_da.iloc[39,1:])  # 시작행 초기화 (코드 실행시 마다)
branch = 1  # 나뭇가지
child = pd.DataFrame()  # 생존한 아이들


while 1:  # 1은 무한반복을 의미

  # 100개의 유전자 생성
  for i in range(100):

    # 교차할 유전자군 (랜덤)
    ge_fa = random.randint(0,2)  # 유전자군 랜덤
    ge_fadata = final_da.loc[final_da.iloc[:,0] == ge_fa,1:].reset_index(drop = True)  # 유전자 군

    # 유전자와 속성 선택
    ge_num = random.randint(0,len(ge_fadata) - 1)  # 행 선택(유전자 선택)
    ge_sel1 = random.randint(1,2)  # 속성 선택
    ge_sel2 = random.randint(3,4)
    ge_sel3 = random.randint(5,6)
    ge_sel = list([ge_sel1, ge_sel2, ge_sel3])
    sel = ge_fadata.loc[ge_num, ge_sel]  # 선택된 행렬에 대한 

    # 정상 유전자 교차
    aaa.loc[ge_sel] = sel.values

    # 돌연변이 생성 (유전자 1회 생성마다 30% 확률로 돌연변이 등장)
    if random.random() <= 0.3:
      mutation1 = random.randint(ge_fadata.loc[:,ge_sel1].min(), ge_fadata.loc[:,ge_sel1].max())
      mutation2 = random.randint(ge_fadata.loc[:,ge_sel2].min(), ge_fadata.loc[:,ge_sel2].max())
      mutation3 = random.randint(ge_fadata.loc[:,ge_sel3].min(), ge_fadata.loc[:,ge_sel3].max())
      mutation = list([mutation1, mutation2, mutation3])
      aaa.loc[ge_sel] = mutation
    
    # 생성된 아이들
    child = child.append(aaa)

  # 생성된 아이들로 모델 예측
  predict = pd.DataFrame(model.predict(child))

  # 살아남은 아이들 랜덤 하나만 생존
  child2 = pd.DataFrame(np.hstack((predict, child)))
  child = child2.loc[child2[0] == k_result, :].iloc[:,1:]  # 원하던 결과랑 같은 것들

  # 나뭇가지 표시
  print('%s branch' % branch)
  branch += 1
  
  # 1000명만 생성하고 전체 반복문 종료
  # print('중복 행: ',child[child.duplicated()])  # 중복행 찾기
  child = child.drop_duplicates()  # 중복 행 제거

  child = child.reset_index(drop = True)

  if len(child) >= 1000:
    break

# 생성된 아이들 확인
child